# This program is created by ...

In [ ]:
# Import all the necessary packages:
import numpy as np                     # type: ignore
import matplotlib.pyplot as plt        # type: ignore
from datasets import load_dataset

%matplotlib inline

In [ ]:
# Download the dataset
ds = load_dataset("marmal88/skin_cancer")


### PRINT ds head() to see first few records?